In [1]:
# initialize PySpark
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.types import *

In [2]:
spark = SparkSession \
    .builder \
    .appName("PySpark SQL Session") \
    .getOrCreate()

In [3]:
# example simple JSON file
df = spark.read.json("example_1.json", multiLine=True)
df.show()

+-----+-----+-----+
|color|fruit| size|
+-----+-----+-----+
|  Red|Apple|Large|
+-----+-----+-----+



In [4]:
df.schema

StructType(List(StructField(color,StringType,true),StructField(fruit,StringType,true),StructField(size,StringType,true)))

In [5]:
df.select("color").show()

+-----+
|color|
+-----+
|  Red|
+-----+



In [6]:
df.printSchema()

root
 |-- color: string (nullable = true)
 |-- fruit: string (nullable = true)
 |-- size: string (nullable = true)



In [7]:
df.explain()

== Physical Plan ==
*(1) FileScan json [color#0,fruit#1,size#2] Batched: false, Format: JSON, Location: InMemoryFileIndex[file:/Users/lrego/Projects/py-projects/pyspark-exercise/map_reduce_examples/exa..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<color:string,fruit:string,size:string>


## Nested JSON structure

Perform data analysis in a nested JSON structure.

https://docs.azuredatabricks.net/_static/notebooks/transform-complex-data-types-python.html  
https://sparkbyexamples.com/spark/spark-sql-structtype-on-dataframe/  
https://stackoverflow.com/questions/37471346/automatically-and-elegantly-flatten-dataframe-in-spark-sql
https://adatis.co.uk/parsing-nested-json-lists-in-databricks-using-python/  
https://spark.apache.org/docs/2.3.0/api/python/pyspark.sql.html  
https://stackoverflow.com/questions/34271398/flatten-nested-spark-dataframe
https://docs.databricks.com/spark/latest/dataframes-datasets/introduction-to-dataframes-python.html

This is a unformatted json example.

In [8]:
df2 = spark.read.json("example_2.json", multiLine=True)

In [9]:
df2.printSchema()

root
 |-- quiz: struct (nullable = true)
 |    |-- maths: struct (nullable = true)
 |    |    |-- q1: struct (nullable = true)
 |    |    |    |-- answer: string (nullable = true)
 |    |    |    |-- options: array (nullable = true)
 |    |    |    |    |-- element: string (containsNull = true)
 |    |    |    |-- question: string (nullable = true)
 |    |    |-- q2: struct (nullable = true)
 |    |    |    |-- answer: string (nullable = true)
 |    |    |    |-- options: array (nullable = true)
 |    |    |    |    |-- element: string (containsNull = true)
 |    |    |    |-- question: string (nullable = true)
 |    |-- sport: struct (nullable = true)
 |    |    |-- q1: struct (nullable = true)
 |    |    |    |-- answer: string (nullable = true)
 |    |    |    |-- options: array (nullable = true)
 |    |    |    |    |-- element: string (containsNull = true)
 |    |    |    |-- question: string (nullable = true)



In [10]:
df2.show()

+--------------------+
|                quiz|
+--------------------+
|[[[12, [10, 11, 1...|
+--------------------+



In [15]:
from pyspark.sql import functions

In [16]:
df2.select(explode("quiz.maths.q1.options").alias("q1")).show()

+---+
| q1|
+---+
| 10|
| 11|
| 12|
| 13|
+---+



In [17]:
df3 = df2.withColumn("subject", functions.lit("maths")) \
         .withColumn('q1 options', explode("quiz.maths.q1.options").alias("q1"))

In [18]:
df3.select("subject","q1 options").show()

+-------+----------+
|subject|q1 options|
+-------+----------+
|  maths|        10|
|  maths|        11|
|  maths|        12|
|  maths|        13|
+-------+----------+

